In [ ]:
from pathlib import Path
from TexSoup import TexSoup, TexNode


In [ ]:
doc_path = Path() / 'tree_example.tex'
doc_content = doc_path.read_text()



In [ ]:
from TexSoup.data import TexExpr, TexText, Token, TexCmd

def expr_replace(expr, old, new):
	
# 	print('REMOVE', old, 'from', expr)
	
	if isinstance(expr, TexCmd):
		
# 		print('ARGS', type(expr.args), expr.args, old in expr.args)
		
	
		for idx, val in enumerate(expr.args):
			print(idx, val)
			if val == old:
				expr.args[idx] = new
				return True
			
# 			print(f'Warning: didnt find item to remove {old} in {expr.args}')
	
		
# 		idx = expr.args.remove(old)
# 		expr.args.insert(idx, new)

	else:
		idx = expr.remove(old)
		expr.insert(idx, new)

		
def expr_remove(expr, old):
	expr.remove(old)
		

def tree_descend(node, parent=None, level=0):
	ind = '	' * level
	tx = str(node).strip().replace('\n', '--')
	
	
	if isinstance(node, TexNode):
# 		print(f'{ind}{node.__class__.__name__} |')
# # 		try:
# 		for child in node.expr.all:
# 			tree_descend(child, node, level+1)
		print('NODE ALERT')
	
	elif isinstance(node, TexExpr):
		
		if isinstance(node, TexText):
			
			if tx:
				print(f'{ind}{node.__class__.__name__} | {tx} ')
			
			if tx.startswith('%'):
				expr_replace(parent, node, '%')
		
		elif isinstance(node, TexCmd):
			
			children = node.args
		
			print(f'{ind}{node.__class__.__name__} | {node.name} {children.__len__()} |')
	
			if node.name in {'comment', 'KL'}:
				expr_remove(parent, node)
			elif node.name in {'kl'}:
				num_args = node.args.__len__()
				if num_args == 0:
					expr_remove(parent, node)
				elif num_args == 1:
					expr_replace(parent, node, node.args[0])
				else:
					raise ValueError(f'Short-circuit cmd has {node.args.__len__()} args: {node} in {parent}')
			
			elif node.name in {'includegraphics'}:
				for a in node.args:
					if a.begin == '{':
						img = ''.join(list(a.args) + list(a.contents))
						print('IMAGE', img)

			
			else:
	
				if children:
					for child in children:
						tree_descend(child, node, level+1)
		
		else:
			children = node.all

			print(f'{ind}{node.__class__.__name__} | {children.__len__()} |')
# 		
		
			if children:
				for child in children:
					tree_descend(child, node, level+1)
# 		else:
		
	
	elif isinstance(node, Token):
		print(f'{ind}{node.__class__.__name__} | {tx}')
		
		if node.startswith('%'):
			expr_replace(parent, node, '%')
	
	else:

		print(f'{ind}{node.__class__.__name__} | {tx}')
		
# 		if node.startswith('%'):
# 			print(f'COMMENT {node.strip()}')
# # 			parent.remove(node)
# 		else:
# 			print(f'NO COMMENT [{node.strip()}]')
		
# 		except Exception as e:
			
# 			print(f'{ind}: {e}')

doc = TexSoup(doc_content)
tree_descend(doc.expr)

In [ ]:
# print('DOC\n', doc)
# print('ORIG\n', doc_content)

def to_lines(d):
	return str(d).split('\n')

for l1, l2 in zip(to_lines(doc_content), to_lines(doc)):
	print(f'{l1:<80}|{l2}')


In [ ]:
from TexSoup.data import TexExpr, TexText, Token, TexCmd



def expr_replace(expr, old, new):
	idx = expr.remove(old)
	expr.insert(idx, new)


def tree_descend_node(node, parent=None, level=0):
	ind = '	' * level
	tx = str(node).strip().replace('\n', '--')
	
	children_nodes = node.children
	print(f'{ind}{node.__class__.__name__} {node.name} | {children_nodes.__len__()} |')
	
# 	for child_node in children_nodes:	
# 		tree_descend_node(child_node, parent=node, level=level+1)
	
	
	child_nodes_and_texts = node.contents
	
	for c in child_nodes_and_texts:
		if isinstance(c, TexNode):
			tree_descend_node(c, parent=node, level=level+1)
			
		elif isinstance(c, TexExpr):
			child_node = TexNode(c)
			child_node.parent = node
			
			tree_descend_node(child_node, parent=node, level=level+1)
		else:
			# TEXT!
			ctx = str(c).strip().replace('\n', '--')
			print(f'{ind}	{c.__class__.__name__}: {ctx}' )
			
			if c.startswith('%'):
				node.expr.remove(c)#, '%REDACTED')
	
	
	
	
doc = TexSoup(doc_content)		
tree_descend_node(doc)

In [ ]:
cpt = doc.caption.expr
cpt

In [ ]:
cpt.args[0]